# GR5242 HW01 Problem 2: Dropout as a form of regularization


**Instructions**: This problem is an individual assignment -- you are to complete this problem on your own, without conferring with your classmates.  You should submit a completed and published notebook to Courseworks; no other files will be accepted.


**Description**: In this exercise we will try to understand regularizing effects of [Dropout](https://jmlr.org/papers/v15/srivastava14a.html) method which was initially introduced in the deep learning context to mitigate overfitting, though we intend to study its behavior as a regularizer in a rather simpler settting.


## Regression
Indeed, linear models correspond to a one layer neural networks with linear activation. Denote $f_{\beta}:\mathbb{R}^p \to \mathbb{R}: x \mapsto \sum_{j=1}^{p} \beta_j x_j$ to represent the output of such network. Given $n$ samples $(x_i,y_i)_{i \leq n} \in (\mathbb{R}^{p} \times \mathbb{R})^n$ we want to regress the response onto the observed covariates using the following MSE loss:

$$ L(\beta) =  \sum_{i=1}^{n} (y_i - \sum_{j=1}^{p} \beta_j x_{ij})^2 $$

In the current atmosphere of deep learning practice, it is rather popular to have moderately large networks in order to learn a task (we will see more on this later in the course). This corresponds to having $p \gg n$ in our setting which allows more flexibility in our linear model. However, in these cases where the model can be too complicated one can use `explicit` regularization to penalize complex models. One way to do so is ridge regression:

$$ \hat{\beta}_{\lambda} = \arg\min_{\beta \in \mathbb{R}^p}{ L(\beta) + \lambda \sum_{j=1}^{p}\beta_j^2 } $$


**Question 1**: Show that $\hat{\beta}_{\lambda} = (X^{T}X + \lambda I_{p \times p})^{-1} X^{T} Y$ where $X = (x_1^{'},\dots,x_n^{'})^{'} \in \mathbb{R}^{n \times p}$ and $Y = (y_1,\dots,y_n)^{'} \in \mathbb{R}^{n}$.  

**Answer**: $$\frac{dL}{dβ}=\frac{d}{dβ}(\sum_{i=1}^{n} (y_i - \sum_{j=1}^{p} \beta_j x_{ij})^2+ \lambda \sum_{j=1}^{p}\beta_j^2)$$
$$=\frac{d}{dβ}((Xβ−Y)^T(Xβ−Y)+\lambda β^Tβ)$$
$$=\frac{d}{dβ}(Y^TY−2β^TX^TY+β^TX^TXβ+λβ^Tβ)$$
$$=−2X^TY+2X^TXβ+2λβ=0$$

Then we got $X^TY=(X^TX+λI)β$ so that $\hat{\beta}_{\lambda} = (X^{T}X + \lambda I_{p \times p})^{-1} X^{T} Y$ where $X = (x_1^{'},\dots,x_n^{'})^{'} \in \mathbb{R}^{n \times p}$ and $Y = (y_1,\dots,y_n)^{'} \in \mathbb{R}^{n}$.


## Dropout

We now present the connection between dropout method and ridge regression (outlined in more detail in [Wager et al.](https://arxiv.org/pdf/1307.1493.pdf))

To recap, dropout randomly drops units along with their input/output connections. We now want to apply this method to our simple setting. Let us define the indicator random variable $I_{ij}$ to be whether the $j$'th neuron is present or not in predicting the response of the $i$'th sample. More explicitly, the ouput of the network for $i$'th sample becomes $\sum_{j=1}^{p} I_{ij}\beta_j x_{ij}$ where

$$ I_{ij} = \begin{cases} 0 & \text{with probability } \phi \\ \frac{1}{1-\phi} & \text{with probability } 1-\phi  \end{cases}, \quad 1 \leq j \leq p,\, 1\leq i \leq n, $$

drawn independently from the training dataset. Note that $\mathbb{E}[I_{ij}] = 1$, thus the output of the network is $f_{\beta}(x_i)$ on average. 



**Question 2**: Write down explicitly the loss function after using the dropout as a function of $I = (I_{ij})_{i \leq n, j \leq p}$ denoted by $L(\beta, I)$.

**Answer**: The loss function after using the dropout as a function of $I = (I_{ij})_{i \leq n, j \leq p}$ is 
$$ L(\beta,I) =  \sum_{i=1}^{n} (y_i - \sum_{j=1}^{p} I_{ij}\beta_j x_{ij})^2 $$

It can be shown that SGD + Dropout is in some sense equivalent to minimizing the loss function $L(\beta,I)$ on average. Related to this point, the following problem justifies why dropout can be thought as a form of regularization. 

**Question 3 (bonus question)**: Suppose the feature matrix $X \in \mathbb{R}^{n \times p}$ have standardized features (norm of each column is one). Show that the solution to the following problem corresponds to a ridge regression with $\lambda = \frac{\phi}{1-\phi}$.
$$ \arg\min_{\beta \in \mathbb{R}^p} \mathbb{E}[L(\beta,I)]$$
where the expectation is over the randomness of indicator random variables.

**Hint**: You can assume that taking derivative can pass through expectation.

**Answer**: Taking the derivative we get:
$$\frac{dL(\beta,I)}{dβ}=\frac{d}{dβ}(\sum_{i=1}^{n} (y_i - \sum_{j=1}^{p} I_{ij}\beta_j x_{ij})^2)$$
$$=\frac{d}{dβ}((XβI−Y)^T(XβI−Y))$$
$$=\frac{d}{dβ}(Y^TY−2Iβ^TX^TY+I^2β^TX^TXβ)$$
$$=−2IX^TY+2I^2X^TXβ+2\sum_{j=1,j\not=i}^{n}\beta_jI_iI_jX_i^TX_j$$
which is equivalent to:
$$\frac{dL(\beta,I)}{dβ}=\frac{dL(\beta)}{dβ}+2\beta (p_1I_2+p_2I_1)diag(X^TX)$$
$$=\frac{dL(\beta)}{dβ}+2\beta (\frac{\phi}{1-\phi}+0)diag(X^TX)$$
Thus, we integrate to have
$$ L(\beta,I) =  \sum_{i=1}^{n} (y_i - \sum_{j=1}^{p} I_{ij}\beta_j x_{ij})^2$$
$$=L(\beta)+\frac{ϕ}{1-\phi}\beta^Tdiag(X^TX)β $$
Then, we can transform the soulution to a ridge regression with λ:
$$ \arg\min_{\beta \in \mathbb{R}^p} \mathbb{E}[L(\beta,I)]=arg\min_{\beta \in \mathbb{R}^p}\mathbb{E}[{ L(\beta)] + \lambda \sum_{j=1}^{p}\beta_j^2diag(X^TX)}$$
$$=arg\min_{\beta \in \mathbb{R}^p}\mathbb{E}[{ L(\beta)] + \lambda \sum_{j=1}^{p}\beta_j^2}$$
$$λ=\frac{ϕ}{1-\phi}$$